<a href="https://colab.research.google.com/github/celinaLind/codebase-rag/blob/main/Codebase_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pygithub langchain langchain-community openai tiktoken pinecone-client langchain_pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successf

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from google.colab import userdata
from pinecone import Pinecone
import os
import tempfile
from github import Github, Repository
from git import Repo
from openai import OpenAI
from pathlib import Path
from langchain.schema import Document
from pinecone import Pinecone

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def clone_repository(repo_url):
  """Clones a GitHub repository to a temporary directory.

  Args:
    repo_url: The URL of the GitHub repository to clone.

  Returns:
    The path to the cloned repository.
  """

  repo_name = repo_url.split('/')[-1] # Extract repository from URL
  repo_path= f"/content/{repo_name}"
  Repo.clone_from(repo_url, str(repo_path))
  return str(repo_path)

In [ ]:
path = clone_repository("https://github.com/celinaLind/chitty-chat")

In [ ]:
print(path)

/content/chitty-chat


In [ ]:
SUPPORTED_EXTENSIONS = {'.py', '.js', '.tsx', '.jsx', '.ipynb', '.java',
                         '.cpp', '.ts', '.go', '.rs', '.vue', '.swift', '.c', '.h'}

IGNORED_DIRS = {'node_modules', 'venv', 'env', 'dist', 'build', '.git',
                '__pycache__', '.next', '.vscode', 'vendor'}

In [ ]:
def get_file_content(file_path, repo_path):
    """
    Get content of a single file.

    Args:
        file_path (str): Path to the file

    Returns:
        Optional[Dict[str, str]]: Dictionary with file name and content
    """
    try:
      # 'with' is a context manager that makes sure that, even
      # if there is an error, the file will be closed automatically
      # after being used

      # the below command reads the file from the path provided and
      # sets the string content to the variable 'content'
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # Get relative path from repo root
        rel_path = os.path.relpath(file_path, repo_path)

        return {
            "name": rel_path,
            "content": content
        }
    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None


def get_main_files_content(repo_path: str):
    """
    Get content of supported code files from the local repository.

    Args:
        repo_path: Path to the local repository

    Returns:
        List of dictionaries containing file names and contents
    """
    files_content = []

    try:
      # this will go through a directory tree (a folder and its subfolders)
      # for each directory it encounters a tuple is returned
      # root : the path of the current directory
      # _ : a list of subdirectory names
      # files : a list of file names in the current directory
        for root, _, files in os.walk(repo_path):
            # Skip if current directory is in ignored directories
            if any(ignored_dir in root for ignored_dir in IGNORED_DIRS):
                continue

            # Process each file in current directory
            for file in files:
                file_path = os.path.join(root, file)
                # check to see if the file extension is supported
                if os.path.splitext(file)[1] in SUPPORTED_EXTENSIONS:
                    file_content = get_file_content(file_path, repo_path)
                    if file_content:
                        files_content.append(file_content)

    except Exception as e:
        print(f"Error reading repository: {str(e)}")

    return files_content

In [ ]:
file_content = get_main_files_content(path)

In [ ]:
file_content

[{'name': 'middleware.ts',
  'content': 'import { clerkMiddleware } from "@clerk/nextjs/server";\n\nexport default clerkMiddleware();\n\nexport const config = {\n  matcher: [\n    // Skip Next.js internals and all static files, unless found in search params\n    "/((?!_next|[^?]*\\\\.(?:html?|css|js(?!on)|jpe?g|webp|png|gif|svg|ttf|woff2?|ico|csv|docx?|xlsx?|zip|webmanifest)).*)",\n    // Always run for API routes\n    "/(api|trpc)(.*)",\n  ],\n};\n'},
 {'name': 'tailwind.config.ts',
  'content': 'import type { Config } from "tailwindcss";\n\nconst config: Config = {\n    darkMode: ["class"],\n    content: [\n    "./pages/**/*.{js,ts,jsx,tsx,mdx}",\n    "./components/**/*.{js,ts,jsx,tsx,mdx}",\n    "./app/**/*.{js,ts,jsx,tsx,mdx}",\n  ],\n  theme: {\n  \textend: {\n  \t\tcolors: {\n  \t\t\tbackground: \'hsl(var(--background))\',\n  \t\t\tforeground: \'hsl(var(--foreground))\',\n  \t\t\tcard: {\n  \t\t\t\tDEFAULT: \'hsl(var(--card))\',\n  \t\t\t\tforeground: \'hsl(var(--card-foreground)

# Embeddings

In [ ]:
def get_huggingface_embeddings(text, model_name="sentence-transformers/all-mpnet-base-v2"):
  model = SentenceTransformer(model_name)
  return model.encode(text)

In [ ]:
# Example embedding
test_text = "This is a test"
test_embedding = get_huggingface_embeddings(test_text)
print(test_embedding)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[-1.10771386e-02 -9.87185612e-02 -2.17310451e-02  9.86817013e-03
 -2.34005079e-02  4.28259820e-02  5.96764907e-02  4.51824851e-02
  5.96456341e-02  2.93849669e-02  6.90463185e-02 -2.58708484e-02
  3.30726504e-02 -3.02912202e-02  2.75240578e-02 -3.60070243e-02
  2.39569172e-02 -9.27300937e-03 -2.16343515e-02  2.40064636e-02
 -6.57190979e-02  2.65391916e-03 -2.84890104e-02 -3.27287167e-02
 -4.46030311e-03  4.69275229e-02 -1.40922861e-02 -2.70123165e-02
  1.89233560e-03 -3.74077670e-02  2.61785444e-02 -3.26690041e-02
  1.65958442e-02 -7.42781907e-02  1.80515985e-06 -2.48814793e-03
  7.24844635e-03 -2.23124512e-02 -4.73723188e-02 -1.51719339e-02
 -3.05177681e-02  3.18568535e-02 -2.17708461e-02  3.81209068e-02
 -1.28691588e-02 -5.58012649e-02  1.57686453e-02 -8.00077524e-03
 -3.26290168e-02  9.60758701e-02  1.55185508e-02  1.75194927e-02
  1.45066122e-03 -4.19370010e-02  7.55117983e-02  2.08015107e-02
  3.11325938e-02  4.65105698e-02  6.31525321e-03  7.89387617e-03
  3.33152935e-02 -4.90306

In [ ]:
# Set the PINECONE_API_KEY as an environment variable
pinecone_api_key = userdata.get("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY'] = pinecone_api_key

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get("PINECONE_API_KEY"),)

# Connect to your Pinecone index
pinecone_index = pc.Index("codebase-rag")

In [ ]:
vectorstore = PineconeVectorStore(index_name="codebase-rag", embedding=HuggingFaceEmbeddings())

In [ ]:
documents = []

for file in file_content:
    doc = Document(
        page_content=f"{file['name']}\n{file['content']}",
        metadata={"source": file['name']}
    )

    documents.append(doc)

# Store files from github repo provided in namespace as embeddings to
# the "codebase-rag" database in pinecone
vectorstore = PineconeVectorStore.from_documents(
    documents=documents,
    embedding=HuggingFaceEmbeddings(),
    index_name="codebase-rag",
    namespace="https://github.com/CoderAgent/SecureAgent"
)

# Perform RAG

1. Get your Groq API Key here

2. Paste your Groq API Key into your Google Colab secrets, and make sure to enable permissions for it

In [ ]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get("GROQ_API_KEY")
)

In [ ]:
# Test query
query = "How are python files parsed?"

# Embed the test query
raw_query_embedding = get_huggingface_embeddings(query)

raw_query_embedding

# Perform vector search to find the information most similar to embedded query
# Feel free to change the "top_k" parameter to be a higher or lower number
top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(),
                                   top_k=5,
                                   include_metadata=True,
                                   namespace="https://github.com/CoderAgent/SecureAgent")

In [ ]:
# Show top_matches retreived data
# top_matches lists the source of the files that are most similar
top_matches

In [1]:
# contexts delves within the files and provides what is within the files
contexts = [item['metadata']['text'] for item in top_matches['matches']]
contexts

NameError: name 'top_matches' is not defined

In [ ]:
# augmented_query puts together the prompt for the AI bot
# using context created from the pinecone files and the query provided by the user
augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

In [ ]:
# system_prompt creates the systems persona
system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
"""

# Persona and augmented prompt are provided to the AI llama version
# and asked to return adequate response
llm_response = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": augmented_query}
    ]
)

response = llm_response.choices[0].message.content

# Putting it all together

In [ ]:
def perform_rag(query, namespace_value):
    raw_query_embedding = get_huggingface_embeddings(query)

    top_matches = pinecone_index.query(vector=raw_query_embedding.tolist(), top_k=5, include_metadata=True, namespace=namespace_value)

    # Get the list of retrieved texts
    contexts = [item['metadata']['text'] for item in top_matches['matches']]

    augmented_query = "<CONTEXT>\n" + "\n\n-------\n\n".join(contexts[ : 10]) + "\n-------\n</CONTEXT>\n\n\n\nMY QUESTION:\n" + query

    # Modify the prompt below as need to improve the response quality
    system_prompt = f"""You are a Senior Software Engineer, specializing in TypeScript.

    Answer any questions I have about the codebase, based on the code provided. Always consider all of the context provided when forming a response.
    """

    llm_response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": augmented_query}
        ]
    )

    return llm_response.choices[0].message.content

In [ ]:
response = perform_rag("How are python files parsed?")

print(response)

TO DO: (to above code)
1. have namespace be provided by user
2.

In [1]:
pip install streamlit pyngrok python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.5 MB/s eta 0:00:00


In [2]:
from threading import Thread
from pyngrok import ngrok
from google.colab import userdata

In [17]:
ngrok_token = userdata.get("NGROK_AUTH_TOKEN")

ngrok.set_auth_token(ngrok_token)

In [22]:
def run_streamlit():
  os.system("streamlit run /content/streamlit_app.py --server.port 5011")

In [59]:
%%writefile streamlit_app.py

## Streamlit webapp
import streamlit as st

def new_repo():
    del st.session_state["repo_url"]
    update_ui()

def update_ui():
    if st.session_state.show_input:
      st.session_state.show_input = False
    else:
      st.session_state

st.title("Raggity")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []
    st.session_state.messages.append({"role": "assistant", "content": "Before we begin please provide the repo you are wanting to ask about?"})
    st.session_state.messages.append({"role": "assistant", "content": "Please provide the full GitHub URL for the repository you wish to use. For example: https://github.com/celinaLind/chitty-chat"})

# Display chat messages from history on app rerun
for messages in st.session_state.messages:
    with st.chat_message(messages["role"]):
        st.markdown(messages["content"])

if "repo_url" not in st.session_state:
    st.session_state.repo_url = ""

# Create two columns
left_column, right_column = st.columns(2)

# Left column: GitHub URL input
with left_column:
    if getRepo := st.chat_input("Enter GitHub URL"):
        with st.chat_message("user"):
            st.markdown(getRepo)
        st.session_state.messages.append({"role": "user", "content": getRepo})
        st.session_state.repo_url = getRepo
        st.session_state.messages.append({"role": "assistant", "content": "Thank you! What questions do you have for " + getRepo + "?"})

# Right column: Chat input
with right_column:
    # React to user input
    if prompt := st.chat_input("Ask something"):
        # Display user message in chat message container
        with st.chat_message("user"):
            st.markdown(prompt)
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})
        response = perform_rag(query=prompt, namespace_value=st.session_state.repo_url) if st.session_state.repo_url != "" else "Please add repository and re-ask question"
        # Display assistant response in chat message container
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            st.markdown(response)
        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content": response})

# def new_repo():
#   del st.session_state["repo_url"]
#   st.session_state.show_input = True

# st.title("Raggity")

# # Initialize chat history
# if "messages" not in st.session_state:
#     st.session_state.messages = []
#     st.session_state.messages.append({"role": "assistant", "content": "Before we begin please provide the repo you are wanting to ask about?"})
#     st.session_state.messages.append({"role": "assistant", "content": "Please provide the full GitHub URL for the repository you wish to use. For example: https://github.com/celinaLind/chitty-chat"})

# # Display chat messages from history on app rerun
# for messages in st.session_state.messages:
#     with st.chat_message(messages["role"]):
#         st.markdown(messages["content"])

# if "show_input" not in st.session_state:
#     st.session_state.show_input = True # Initially show the input

# if st.session_state.show_input:
#     if getRepo := st.chat_input("Enter GitHub URL", key="repo_url"):
#       with st.chat_message("user"):
#           st.markdown(getRepo)
#       # Add user message to chat history
#       st.session_state.messages.append({"role": "user", "content": getRepo})
#       st.session_state.messages.append({"role": "assistant", "content": "Thank you! What questions do you have for " + getRepo + "?"})
#       st.session_state.show_input = False # Hide input after URL is entered
# else:
#   st.button("New Repo", on_click=new_repo)
#   # React to user input
#   if prompt := st.chat_input("Ask something"):
#       # Display user message in chat message container
#       with st.chat_message("user"):
#           st.markdown(prompt)
#       # Add user message to chat history
#       st.session_state.messages.append({"role": "user", "content": prompt})
#       response = perform_rag(query=prompt, namespace_value=st.session_state.repo_url)
#       # Display assistant response in chat message container
#       with st.chat_message("assistant"):
#           message_placeholder = st.empty()
#           st.markdown(response)
#       # Add assistant response to chat history
#       st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting streamlit_app.py


In [49]:
# Start Streamlit server on a specific port
!nohup streamlit run --no-reload streamlit_app.py --server.port 5011 &

# Start ngrok tunnel to expose the Streamlit server
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)

# Print the URL of the ngrok tunnel
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://9f3e-34-23-62-16.ngrok-free.app


In [48]:
!nohup streamlit --version

nohup: ignoring input and appending output to 'nohup.out'
